In [ ]:
!pip install langchain # langchain 활용
!pip install openai==0.27.8 # chatgpt api 활용
!pip install google-search-results # Google Search 결과를 가져오기 위한 라이브러리
!pip install wikipedia # Wikipedia API를 사용하기 위한 라이브러리
!pip install faiss-cpu # FAISS는 Facebook에서 만든 오픈소스 벡터 데이터베이스 라이브러리 (MIT license)
!pip install sentence_transformers # HuggingFace Embedding 사용 위해서 필요
!pip install tiktoken # 텍스트를 요약할 때 필요한 라이브러리
!pip install PyPDF2 # PDF 파일을 읽어오기 위한 라이브러리

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('')

In [ ]:
#@markdown https://serpapi.com/manage-api-key
#@markdown 구글 검색하기 위해서 필요 (월 100회 무료)
#serpapi에서 발급받은 api 키를 입력해 주세요
SERPAPI_API_KEY = "your_api_key" #@param {type:"string"}

In [ ]:
#@markdown https://huggingface.co/settings/tokens
#@markdown HuggingFace에서 모델 다운로드나 클라우드 모델 사용하기 위해서 필요 (무료)
# huggingFace api 키를 입력해 주세요
HUGGINGFACEHUB_API_TOKEN = "your_api_key" #@param {type:"string"}

In [ ]:
pip install langchain_community

In [ ]:
#chatgpt 기본 사용법
from langchain.llms import OpenAI
import langchain_community


In [ ]:
llm = OpenAI(model_name = 'gpt-3.5-turbo', temperature=0.9)
llm('90년대 kpop 히트곡을 알려줘')

In [ ]:
#prompt 사용하기
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.9)
sys = SystemMessage(content='당신은 영화 추천을 해주는 전문 AI입니다.')
msg = HumanMessage(content='2000년대 개봉한 액션 영화 5개를 추천해줘')
aimsg = chat([sys, msg])
aimsg


In [ ]:
#prompt template / chain 사용

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables = ['상품'],
    template = '{상품} 기반으로 재미있는 메뉴 이름을 추천해줘',
)

prompt.format(상품='AI 치킨 추천서비스')

In [ ]:
from langchain.chains import LLMChain

chat = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature=0.9)
chain = LLMChain(llm = chat, prompt = prompt)

chain.run(상품='치킨 추천')

ChatPromptTemplate / chain 번역하기

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
ChatPromptTemplate,
SystemMessagePromptTemplate,
HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = 'You are a helpful assistant that translates {input_language} to {output_language}.'
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chatchain = LLMChain(llm = chat, prompt=chat_prompt)
chatchain.run(input_language="English", output_language="Korean", text="I love Deep-Learning")

Agent 와 Tool 검색결과 확인하기

In [ ]:
#검색 결과 길반으로 계산하기 (구글대표와 테슬라 대표 몇살 차이?)

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

# tools =load_tools(["serpapi", "llm-math"], llm=chat)
tools =load_tools(["wikipedia", "llm-math"], llm=chat)
agent = initialize_agent(tools, llm=chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

#tools >> 특정 일을 하는 존재 예) 구글검색, 위키피디아 검색, db조회, 수학계산 등
#agent >> 어떤 도구(tools)를 사용, 어떤 순서로 실행할지 결정하는 존재(LLM 이용)

print(agent.run("Google CEO와 Tesla CEO는 각각 몇살이고 이 두사람의 나이 차이는 몇 살 차이야?"))
print()
print(agent.tools)
print(agent.tools[0].description)
print(agent.tools[1].description)


웹페이지 내용 요약

In [ ]:
# https://ko.wikipedia.org/wiki/NewJeans
# 뉴진스 설명 웹페이지

# Document Loaders

from langchain.document_loaders import WebBaseLoader

web_site = "위키피디아 url"

#웹페이지 불러오기
loader = WebBaseLoader(web_path=web_site)
document = loader.load()


In [ ]:
#TextSplitter
#웹페이지 내용이 많기 때문에 chatgpt가 이해하기 쉽게 텍스트 분할
#(현실적으로) 과소비 주의!

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(document)

In [ ]:
print(len(docs))

In [ ]:
print(docs[0])

In [ ]:
print(docs[10])

In [ ]:
#요약하기
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(chat, chain_type="map_reduce", verbose=True)
chain.run(docs[:3])

In [ ]:
# map_reduce란, 나눠서 요약하고 전체본 요약을 다시해서 합치는 과정
# stuff : 한번에 다 넣고 요약 // 오류날 확률이 높음
# refine : 요약 + 다음 문서 >> 요약
# map_rerank : 점수를 매겨서 중요한 거 요약
# verbose=True : 진행과정을 보여줌

Embeddings and VectorStore 활용, 학습시키기

In [ ]:
#vector db로컬에 저장

#embedding and vetorsotre
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings()
# embedding이란?
# LLM에 input(text) 넣었을 때 나오는 특징 숫자열(벡터)
# 보통 512개의 float로 이루어져 있음
# >> 의미상 비슷한 input이 들어갔을 때, 나오는 특징벡터가 유사한 특징을 갖고, 거리상 가깝다.
# 예) 바나나/사과

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

index = VectorstoreIndexCreator(
    vectorstore_cls = FAISS,
    embedding = embeddings,

).from_loaders([loader])

#파일로 저장
index.vectorstore.save_local("faiss-nj")

In [ ]:
#vector-db 에 질문하기
index.query("뉴진스의 첫 노래는?", llm=chat, verbose=True)

In [ ]:
index.query("뉴진스 구성원 수는?", llm=chat, verbose=True)


FAISS 벡터 DB 디스크에서 불러오기

In [ ]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

fdb = FAISS.load_local("faiss-nj", embeddings, allow_dangerous_deserialization=True)
index2 = VectorStoreIndexWrapper(vectorstore=fdb)

In [ ]:
index2.query("뉴진스의 멤버는 누구야?", llm=chat, verbose=False)